<a href="https://colab.research.google.com/github/mohammadhosseinipour/Mistral/blob/main/notebooks/Mixtral_7*8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()


In [2]:
!pip install --upgrade google-cloud-storage


In [3]:
!gcloud config set project exalted-tape-424823-q5


Updated property [core/project].


In [4]:
!gcloud auth list
!gcloud beta billing projects describe exalted-tape-424823-q5

       Credentialed Accounts
ACTIVE  ACCOUNT
*       m.hosseinipour.78@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`

billingAccountName: billingAccounts/012064-7BFB05-E8421E
billingEnabled: true
name: projects/exalted-tape-424823-q5/billingInfo
projectId: exalted-tape-424823-q5


In [1]:
!pip install hqq


In [2]:
# fix numpy in colab
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()

In [3]:
# Step 1: Prepare Mixtral model

# Install and import libraries
import sys
import os

# Assuming 'src' is in the same directory as your current working directory in Colab
# Adjust the path accordingly if 'src' is located elsewhere
sys.path.append(os.path.abspath('./mixtral-offloading'))

import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
from transformers import TextStreamer

# Fix numpy in colab
import numpy
# Add the path where 'src' is located
sys.path.append(os.path.abspath('./mixtral-offloading'))
from src.build_model import OffloadConfig, QuantConfig, build_model

clear_output()


Login to huggingface

In [4]:
from huggingface_hub import notebook_login
notebook_login()

initialize model

In [5]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

In [7]:
import requests
from bs4 import BeautifulSoup

def get_rag_input_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

rag_input_data = get_rag_input_data("https://www.unipd.it/en/how-apply")

In [8]:
# Step 3: Segment the content

def chunk_text(text, chunk_size=512):
    sentences = text.split('. ')
    chunks = []
    current_chunk = []
    current_length = 0
    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length > chunk_size:
            chunks.append('. '.join(current_chunk) + '.')
            current_chunk = [sentence]
            current_length = sentence_length
        else:
            current_chunk.append(sentence)
            current_length += sentence_length
    if current_chunk:
        chunks.append('. '.join(current_chunk) + '.')
    return chunks

chunks = chunk_text(rag_input_data)

In [9]:
# Function to integrate content into the model's context
def integrate_content_into_context(model, tokenizer, content_chunks):
    max_input_length = model.config.max_position_embeddings
    for chunk in content_chunks:
        input_ids = tokenizer(chunk, return_tensors='pt', truncation=True, max_length=max_input_length).input_ids.to(device)
        attention_mask = torch.ones_like(input_ids)
        with torch.no_grad():
            model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                past_key_values=None,
                max_new_tokens=1  # Setting this to a minimal value
            )
# Integrate content into the model
integrate_content_into_context(model, tokenizer, chunks)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
# Step 5: Perform the Evaluation

questions = [
    "Can I apply for the admission of the university of Padua if I am a non-italian student?",
    "Are the courses provided by the university of Padua taught only in Italian?",
    "I missed the deadlines for submitting application for the first semester. Can I enroll and submit the application for the second (winter) semester?",
    "I want to apply for the bachelor's degree in biology, how to know if I am eligible?",
    "The duration of diploma that I obtained is 11 years in my country, does that mean that I can not apply for the admission of the bachelor's degree program in biology of the university of Padua?",
    "How can I make sure if my previous university which is not Italian is qualified in order to apply for the master's degree programs?",
    "Can I apply for just single course units not the whole degree programs of the University of Padua?",
    "My Bachelor's degree was a double degree program obtained from two different universities, one from Indonesia and one from Singapore, how does this affect my application for admission for the master's degree programs?",
    "What are the eligibilities to apply for the admission to a master's degree program?",
    "Can you give the contact info so I can get more detailed academic information?"
]

llm_outputs = []

for question in questions:
    user_entry = dict(role="user", content=question)
    input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

    if past_key_values is None:
        attention_mask = torch.ones_like(input_ids)
    else:
        seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
        attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

    result = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        past_key_values=past_key_values,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
        max_new_tokens=512,  # Adjusted to control output length
        pad_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
        output_hidden_states=True,
    )

    answer = tokenizer.decode(result["sequences"][0], skip_special_tokens=True)
    llm_outputs.append(answer.strip())

# Ensure we have 10 responses
print(len(llm_outputs))
print(llm_outputs)